## Exercise: Pandas & NumPy with Spotify Dataset
Use pandas and NumPy to analyze, filter, manipulate, and visualize data from the Spotify 2023 dataset.

### Task 1: Data exploration and cleaning
1. Load the dataset in pandas.
2. Check for missing values and handle them:
    - Replace missing values in the "key" column with "Unknown".
    - Fill missing values in "in_shazam_charts" with 0.
3. Filter the dataset:
    - Extract all tracks from 2023 that have been in Spotify Charts at least 50 times.
    - Save this subset as "popular_tracks_2023.csv".

In [4]:
import pandas as pd

spotify_df = pd.read_csv("spotify-2023.csv", encoding_errors="ignore")

In [5]:
print(spotify_df.isnull().sum())

# 2. Handle missing values
spotify_df["key"].fillna("Unknown", inplace=True)
spotify_df["in_shazam_charts"].fillna(0, inplace=True)

# 3. Filter the dataset: Extract tracks from 2023 that have been in Spotify Charts at least 50 times
filtered_df = spotify_df[(spotify_df["released_year"] == 2023) & (spotify_df["in_spotify_charts"] >= 50)]
filtered_df.to_csv("popular_tracks_2023.csv")

track_name               2
artist(s)_name           0
artist_count             0
released_year            0
released_month           0
released_day             0
in_spotify_playlists     0
in_spotify_charts        0
streams                  0
in_apple_playlists       0
in_apple_charts          0
in_deezer_playlists      0
in_deezer_charts         0
in_shazam_charts        50
bpm                      0
key                     95
mode                     0
danceability_%           0
valence_%                0
energy_%                 0
acousticness_%           0
instrumentalness_%       0
liveness_%               0
speechiness_%            0
dtype: int64


In [6]:
spotify_df["streams"] = pd.to_numeric(spotify_df["streams"], errors='coerce')

# Check if there are any NaN values now
print(spotify_df["streams"].isna().sum())

1


### Task 2: Statistical analysis and aggregation
1. Calculate basic statistics:
    - Find the average BPM (tempo) by key.
    - Find the average energy level for songs with more than 100 million streams.
2. Sort the dataset:
    - Find the top 10 most streamed songs.
    - Find the 5 least danceable songs.
3. Group the dataset:
    - Count how many tracks belong to each mode (Major/Minor).

In [7]:
# 1. Find the average BPM by key
avg_bpm_by_key = spotify_df.groupby("key")["bpm"].mean()
print("Average BPM by Key:\n", avg_bpm_by_key)

# One row has weird values, exclude it for now
spotify_df["streams"] = pd.to_numeric(spotify_df["streams"], errors='coerce')

# Find the average energy level for songs with more than 100 million streams
avg_energy = spotify_df[spotify_df["streams"] > 100000000]["energy_%"].mean()
print(f"Average Energy Level for Songs with More than 100M Streams: {round(avg_energy, 2)}")

Average BPM by Key:
 key
A          127.840000
A#         119.719298
B          121.543210
C#         122.341667
D          123.802469
D#         123.393939
E          121.935484
F          120.235955
F#         125.479452
G          122.208333
G#         123.021978
Unknown    119.947368
Name: bpm, dtype: float64
Average Energy Level for Songs with More than 100M Streams: 64.11


In [8]:
# 2. Find the top 10 most streamed songs
top_10_songs = spotify_df.nlargest(10, "streams")[["track_name", "artist(s)_name", "streams"]].reset_index(drop = True)
top_10_songs.index = top_10_songs.index + 1 
print("Top 10 Most Streamed Songs:\n", top_10_songs)

# Find the 5 least danceable songs
least_danceable_songs = spotify_df.nsmallest(5, "danceability_%")[["track_name", "artist(s)_name", "danceability_%"]].reset_index(drop = True)
least_danceable_songs.index = least_danceable_songs.index + 1
print("5 Least Danceable Songs:\n", least_danceable_songs)

Top 10 Most Streamed Songs:
                                        track_name  \
1                                 Blinding Lights   
2                                    Shape of You   
3                               Someone You Loved   
4                                    Dance Monkey   
5   Sunflower - Spider-Man: Into the Spider-Verse   
6                                       One Dance   
7                       STAY (with Justin Bieber)   
8                                        Believer   
9                                          Closer   
10                                        Starboy   

                  artist(s)_name       streams  
1                     The Weeknd  3.703895e+09  
2                     Ed Sheeran  3.562544e+09  
3                  Lewis Capaldi  2.887242e+09  
4                    Tones and I  2.864792e+09  
5          Post Malone, Swae Lee  2.808097e+09  
6            Drake, WizKid, Kyla  2.713922e+09  
7   Justin Bieber, The Kid Laroi  2.665344e+

In [9]:
# 3. Count how many tracks belong to each mode (Major/Minor)
mode_counts = spotify_df["mode"].value_counts()
mode_counts.columns = ["Mode", "Track Count"]
print("Number of Tracks by Mode:\n", mode_counts)

Number of Tracks by Mode:
 mode
Major    550
Minor    403
Name: count, dtype: int64


### Task 3: Feature engineering
1. Create a new variable "track_popularity" using the following logic:
    - "Super Hit" if streams > 500M.
    - "Hit" if streams between 100M and 500M.
    - "Moderate" if streams between 50M and 100M.
    - "Less Popular" otherwise.
2. Save the modified dataset with the new column as "track_popularity_data.csv".

In [10]:
def classify_popularity(streams):
    if streams > 500000000:
        return "Super Hit"
    elif streams >= 100000000 and streams <= 500000000:
        return "Hit"
    elif streams >= 50000000 and streams < 100000000:
        return "Moderate"
    else:
        return "Less Popular"

In [11]:
spotify_df["track_popularity"] = spotify_df["streams"].apply(classify_popularity)
spotify_df.to_csv("track_popularity_data.csv")

### Task 4: Unique Task
Each student must create their own unique variable in the dataset. Choose one approach:
1. Assign a playlist ranking (playlist_rank):
    - Generate a random rank between 1 and 100 for each track using np.random.randint().
2. Create an emotional category (mood_category):
    - Categorize songs based on valence (happiness) and energy.
      - "Energetic & Happy" if valence > 60 and energy > 70.
      - "Calm & Happy" if valence > 60 and energy <= 70.
      - "Sad" if valence < 40.
      - "Neutral" otherwise.
3. Define a personal popularity score (custom_popularity_score):
    - Use a custom formula (e.g., (streams / bpm) * danceability_%).

In [12]:
import numpy as np
        
# 1. Assign playlist ranking:
spotify_df["playlist_rank"] = np.random.randint(1, 101, size=len(spotify_df))

In [13]:
# helper function:
def classify_mood(valence, energy):
    if valence > 60 and energy > 70:
        return "Energetic & Happy"
    elif valence > 60 and energy <= 70:
        return "Calm & Happy"
    elif valence < 40:
        return "Sad"
    else:
        return "Neutral"
        
# 2. Create an emotional category:
spotify_df["mood_category"] = spotify_df.apply(lambda x: classify_mood(x["valence_%"], x["energy_%"]), axis=1)

In [14]:
# Normalize streams to 0 (popular) to 1 (not so popular)
spotify_df["normalized_streams"] = 1 - (spotify_df["streams"] - spotify_df["streams"].min()) / (spotify_df["streams"].max() - spotify_df["streams"].min())

spotify_df["custom_popularity_score"] = (spotify_df["normalized_streams"]*3) + (10/spotify_df["valence_%"]) + spotify_df["speechiness_%"]/20 + spotify_df["acousticness_%"]/50

my_top_10_songs = spotify_df.sort_values(by="custom_popularity_score", ascending=False).head(10)

print(my_top_10_songs[["track_name", "artist(s)_name", "streams", "valence_%", "custom_popularity_score"]])

# Well, at least got some kendrick songs

                                            track_name  \
247                                          Cart��o B   
825                                            Flowers   
259                          MIENTRAS ME CURO DEL CORA   
809  Mother I Sober (feat. Beth Gibbons of Portishead)   
341                                             LET GO   
802                                     Auntie Diaries   
783                                 Worldwide Steppers   
780                                             Savior   
848                         Die Young (feat. 347aidan)   
777                                             Andrea   

                         artist(s)_name      streams  valence_%  \
247              MC Caverinha, KayBlack   71573339.0         55   
825                Lauren Spencer Smith  184826429.0         31   
259                             Karol G  206399629.0         57   
809        Kendrick Lamar, Beth Gibbons   33381454.0         50   
341                       